# Convert annotations to MS Coco format with more meta fields

Convert datasets: train 500k, validation and test

We added into annotations:
- 'IsOccluded'
- 'IsTruncated'
- 'IsDepiction'
- 'IsInside'

Apply prefiltering: 
- images with any dimension larger 2000 pixels are ignored
- do not write annotations without bboxes

In [1]:
# data_path="/home/data"
# !cd ../ && ln -s {data_path} input
!ls ../input/

as_mscoco
challenge2018
class-descriptions-boxable.csv
lost+found
test
test-annotations-bbox.csv
test-annotations-human-imagelabels-boxable.csv
train
train-annotations-bbox.csv
train-annotations-human-imagelabels-boxable.csv
train-images-boxable-with-rotation.csv
validation
validation-annotations-bbox.csv
validation-annotations-human-imagelabels-boxable.csv


In [2]:
!ls ../input/train | wc -l
!ls ../input/test | wc -l
!ls ../input/validation | wc -l

505563
125436
41620


In [3]:
try:
    from pathlib import Path
except ImportError:
    from pathlib2 import Path

import numpy as np
import pandas as pd

import json
from PIL import Image

import matplotlib.pylab as plt
import seaborn as sns
%matplotlib inline

/usr/local/lib/python2.7/dist-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/usr/local/lib/python2.7/dist-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/usr/local/lib/python2.7/dist-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/usr/local/lib/python2.7/dist-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/usr/local/lib/python2.7/dist-packages/pandas/core/indexes/base.py:7: RuntimeWarning: numpy.dtype size changed, may indicate bi

In [4]:
TRAIN_IMAGES_PATH = Path(".").resolve().parent / "input" / "train"
TRAIN_ANNOTATIONS_CSV_PATH = Path(".").resolve().parent / "input" / "train-annotations-bbox.csv"
TRAIN_CONFIDENCE_CSV_PATH = Path(".").resolve().parent / "input" / "train-annotations-human-imagelabels-boxable.csv"
TRAIN_IMGINFO_CSV_PATH = Path(".").resolve().parent / "input" / "train-images-with-rotation.csv"
VALIDATION_IMAGES_PATH = Path(".").resolve().parent / "input" / "validation"
VALIDATION_ANNOTATIONS_CSV_PATH = Path(".").resolve().parent / "input" / "validation-annotations-bbox.csv"
VALIDATION_CONFIDENCE_CSV_PATH = Path(".").resolve().parent / "input" / "validation-annotations-human-imagelabels-boxable.csv"
VALIDATION_IMGINFO_CSV_PATH = Path(".").resolve().parent / "input" / "validation-images-with-rotation.csv"
LABELS_DESCRIPTION_CSV_PATH = Path(".").resolve().parent / "input" / "class-descriptions-boxable.csv"

In [5]:
TEST_IMAGES_PATH = Path(".").resolve().parent / "input" / "test"
TEST_ANNOTATIONS_CSV_PATH = Path(".").resolve().parent / "input" / "test-annotations-bbox.csv"
TEST_CONFIDENCE_CSV_PATH = Path(".").resolve().parent / "input" / "test-annotations-human-imagelabels-boxable.csv"

In [6]:
labels_description = pd.read_csv(LABELS_DESCRIPTION_CSV_PATH, header=None)
labels = labels_description[1].values.tolist()

coco_categories = []
for i, label in enumerate(labels):
    coco_categories.append({
        'id': i,
        'name': label,
        'supercategory': label
    })
    
categories = {}
for d in coco_categories:
    categories[d['name']] = d['id']    

In [7]:
xyxy_cols = ['XMin', 'YMin', 'XMax', 'YMax']
meta_cols = ['IsOccluded', 'IsTruncated', 'IsGroupOf', 'IsDepiction', 'IsInside']
ignore_is_crowd = True


def get_bboxes_labels_meta(canvas_size, image_id, annotations):
    bboxes = annotations.loc[image_id, xyxy_cols].values
    labels = annotations.loc[image_id, 'LabelName']
    meta = annotations.loc[image_id, meta_cols].values
    
    if bboxes.ndim == 1:
        bboxes = bboxes[None, :]
        meta = meta[None, :]
    
    if isinstance(labels, str):        
        labels = np.array([labels, ])
        
    # BBox format should be (x, y, w, h)
    bboxes[:, 0] *= canvas_size[0]
    bboxes[:, 1] *= canvas_size[1]
    
    bboxes[:, 2] *= canvas_size[0]
    bboxes[:, 2] -= bboxes[:, 0]
    
    bboxes[:, 3] *= canvas_size[1]
    bboxes[:, 3] -= bboxes[:, 1]
    return bboxes, labels, meta


def compute_area(bbox):
    return bbox[2] * bbox[3]


In [11]:
import tqdm


def create_annotations_json(images_path, annotations, coco_categories, output_mode, 
                            image_ids=None, ignore_is_crowd=True):
    
    coco_images = []
    coco_annotations = []
    
    if image_ids is None:
        image_ids = annotations.index.unique()        

        
    for image_id in tqdm.tqdm(image_ids):
        
        p = images_path / "{}.jpg".format(image_id)
        if not p.exists():
            continue
        
        img = Image.open(images_path / "{}.jpg".format(image_id))

        if max(img.size) > 2000 or min(img.size) < 100:
            continue

        image_info = {
                "id": image_id,
                "file_name": "{}.jpg".format(image_id),
                "width": img.size[0],
                "height": img.size[1],
        }    
        
        bboxes, labels, meta = get_bboxes_labels_meta(img.size, image_id, annotations)

        if len(bboxes) == 0:
            print("No bboxes for image_id '{}'".format(image_id))
            continue

        coco_images.append(image_info)        
   
        for i, (bbox, label, m) in enumerate(zip(bboxes, labels, meta)):
            m = [int(v) for v in m]
            bbox = [int(v) for v in bbox.tolist()]
            annotation_id = hash(image_id + "_{}".format(i))
            annotation_info = {
                "id": annotation_id,
                "image_id": image_id,
                "category_id": categories[label],
                "IsOccluded": m[0],
                "IsTruncated": m[1],
                "iscrowd": m[2] if not ignore_is_crowd else 0,
                "IsDepiction": m[3],
                "IsInside": m[4],            
                "area": int(compute_area(bbox)),
                "bbox": bbox,
                "segmentation": [],
            } 
            coco_annotations.append(annotation_info)  

    output_coco_annotations = {
        "categories": coco_categories,
        "images": coco_images,
        "annotations": coco_annotations
    }
    
    output_folder = Path(".").resolve().parent / "input" / "as_mscoco" / "annotations" 
    if not output_folder.exists():
        output_folder.mkdir(parents=True)
    
    with open((output_folder / "{}.json".format(output_mode)).as_posix(), 'w') as h:
        json.dump(output_coco_annotations, h)    

Create train dataset:

In [12]:
images_path = TRAIN_IMAGES_PATH
annotations_path = TRAIN_ANNOTATIONS_CSV_PATH
output_mode = "train"

annotations = pd.read_csv(annotations_path, index_col="ImageID")
annotations['LabelName'] = annotations['LabelName'].map(labels_description.set_index(0)[1])

In [13]:
create_annotations_json(images_path, annotations, coco_categories, output_mode, 
                        ignore_is_crowd=ignore_is_crowd)


  0%|          | 4048/1743042 [00:15<1:49:13, 265.34it/s]


  0%|          | 8356/1743042 [00:29<1:40:41, 287.14it/s]


  1%|          | 12692/1743042 [00:42<1:37:12, 296.65it/s]


  1%|          | 17046/1743042 [00:56<1:35:15, 302.00it/s]


  1%|          | 21492/1743042 [01:10<1:33:41, 306.27it/s]


...

 29%|██▉       | 505212/1743042 [57:09<2:20:03, 147.30it/s]


 43%|████▎     | 757149/1743042 [57:24<1:14:44, 219.82it/s]


 67%|██████▋   | 1159282/1743042 [57:37<29:01, 335.27it/s]


 89%|████████▉ | 1559581/1743042 [57:51<06:48, 449.31it/s]


100%|██████████| 1743042/1743042 [57:57<00:00, 501.27it/s]

Create validation dataset:

In [15]:
images_path = VALIDATION_IMAGES_PATH
annotations_path = VALIDATION_ANNOTATIONS_CSV_PATH
output_mode = "val"

annotations = pd.read_csv(annotations_path, index_col="ImageID")
annotations['LabelName'] = annotations['LabelName'].map(labels_description.set_index(0)[1])

In [16]:
create_annotations_json(images_path, annotations, coco_categories, output_mode, 
                        ignore_is_crowd=ignore_is_crowd)


  3%|▎         | 1146/35925 [00:18<09:20, 62.04it/s]


  7%|▋         | 2411/35925 [00:37<08:34, 65.14it/s]


 10%|█         | 3624/35925 [00:56<08:19, 64.70it/s]


 14%|█▎        | 4916/35925 [01:14<07:48, 66.26it/s]


 17%|█▋        | 6284/35925 [01:33<07:20, 67.36it/s]


...

 88%|████████▊ | 31678/35925 [07:36<01:01, 69.39it/s]


 92%|█████████▏| 32929/35925 [07:55<00:43, 69.32it/s]


 95%|█████████▌| 34150/35925 [08:13<00:25, 69.27it/s]


 99%|█████████▊| 35434/35925 [08:31<00:07, 69.33it/s]


100%|██████████| 35925/35925 [08:37<00:00, 69.36it/s]

Create test dataset:

In [17]:
images_path = TEST_IMAGES_PATH
annotations_path = TEST_ANNOTATIONS_CSV_PATH
output_mode = "test"

annotations = pd.read_csv(annotations_path, index_col="ImageID")
annotations['LabelName'] = annotations['LabelName'].map(labels_description.set_index(0)[1])

In [18]:
create_annotations_json(images_path, annotations, coco_categories, output_mode, 
                        ignore_is_crowd=ignore_is_crowd)


  1%|          | 1204/108159 [00:17<26:32, 67.17it/s]


  2%|▏         | 2464/108159 [00:35<25:30, 69.08it/s]


  3%|▎         | 3779/108159 [00:53<24:45, 70.26it/s]


  5%|▍         | 5086/108159 [01:11<24:15, 70.81it/s]


  6%|▌         | 6283/108159 [01:30<24:20, 69.76it/s]


...

 96%|█████████▌| 103976/108159 [24:53<01:00, 69.64it/s]


 97%|█████████▋| 105227/108159 [25:10<00:42, 69.65it/s]


 98%|█████████▊| 106529/108159 [25:28<00:23, 69.71it/s]


100%|█████████▉| 107864/108159 [25:45<00:04, 69.79it/s]


100%|██████████| 108159/108159 [25:49<00:00, 69.81it/s]

Create train dataset to check overfitting

- 10 images from test

In [21]:
images_path = TEST_IMAGES_PATH
annotations_path = TEST_ANNOTATIONS_CSV_PATH
output_mode = "train_overfit"

annotations = pd.read_csv(annotations_path, index_col="ImageID")
annotations['LabelName'] = annotations['LabelName'].map(labels_description.set_index(0)[1])
image_ids = annotations.index.unique()
image_ids = image_ids[:10]

In [22]:
create_annotations_json(images_path, annotations, coco_categories, output_mode, 
                        ignore_is_crowd=ignore_is_crowd, image_ids=image_ids)

100%|██████████| 10/10 [00:00<00:00, 98.99it/s]


Create symlinks

In [12]:
!ls -all ../input/as_mscoco/

total 16
drwxr-xr-x 4 root root 4096 Jul 28 14:58 .
drwxrwxrwx 7 1000 1000 4096 Jul 28 11:10 ..
drwxr-xr-x 2 root root 4096 Jul 28 11:22 annotations
drwxr-xr-x 2 root root 4096 Jul 28 11:22 train_overfit


In [19]:
output_mode = "train"
images_path = TRAIN_IMAGES_PATH

In [20]:
output_images_folder = Path(".").resolve().parent / "input" / "as_mscoco" / output_mode
if not output_images_folder.exists():
    output_images_folder.symlink_to(images_path, target_is_directory=True)

In [13]:
output_mode = "val"
images_path = VALIDATION_IMAGES_PATH

In [14]:
output_images_folder = Path(".").resolve().parent / "input" / "as_mscoco" / output_mode
if not output_images_folder.exists():
    output_images_folder.symlink_to(images_path, target_is_directory=True)

In [15]:
output_mode = "test"
images_path = TEST_IMAGES_PATH

In [16]:
output_images_folder = Path(".").resolve().parent / "input" / "as_mscoco" / output_mode
if not output_images_folder.exists():
    output_images_folder.symlink_to(images_path, target_is_directory=True)

In [47]:
output_mode = "train_overfit"

In [48]:
output_images_folder = Path(".").resolve().parent / "input" / "as_mscoco" / output_mode

if not output_images_folder.exists():
    output_images_folder.mkdir()

for image_id in image_ids:
    !ln -s {images_path.as_posix()}/{image_id}.jpg {output_images_folder}/{image_id}.jpg 


In [49]:
!ls {output_images_folder}

000026e7ee790996.jpg  0002ab0af02e4a77.jpg  00045d609ca3f4eb.jpg
000062a39995e348.jpg  0002cc8afaf1b611.jpg  00068d5450f0358b.jpg
0000c64e1253d68f.jpg  0003d84e0165d630.jpg
000132c20b84269b.jpg  000411001ff7dd4f.jpg


In [21]:
!ls -all ../input/as_mscoco/

total 16
drwxr-xr-x 4 root root 4096 Aug  2 06:31 .
drwxrwxrwx 8 1000 1000 4096 Jul 28 18:47 ..
drwxr-xr-x 2 root root 4096 Aug  2 00:27 annotations
lrwxrwxrwx 1 root root   31 Jul 28 14:58 test -> /home/project/oiv4od/input/test
lrwxrwxrwx 1 root root   32 Aug  2 06:31 train -> /home/project/oiv4od/input/train
drwxr-xr-x 2 root root 4096 Jul 28 11:22 train_overfit
lrwxrwxrwx 1 root root   37 Jul 28 14:58 val -> /home/project/oiv4od/input/validation


Test with pycocotools

In [19]:
from pycocotools import coco

In [20]:
output_folder = Path(".").resolve().parent / "input" / "as_mscoco" / "annotations" 

coco = coco.COCO((output_folder / "test.json").as_posix())

loading annotations into memory...
Done (t=6.48s)
creating index...
index created!


In [22]:
image_ids = coco.getImgIds()
image_ids.sort()
roidb = coco.loadImgs(image_ids)

In [25]:
type(roidb), len(roidb) * 2

(list, 215976)

In [16]:
coco.loadImgs(['5840d582ce4fbe93', ])

[{'file_name': '5840d582ce4fbe93.jpg',
  'height': 683,
  'id': '5840d582ce4fbe93',
  'width': 1024}]

Check complete datasets on errors:
- no annotations, 
- annotation has zero or negative size
- annotation is out of bounds

In [13]:
import json

output_folder = Path(".").resolve().parent / "input" / "as_mscoco" / "annotations" 
annotations_file = output_folder / "test.json"

with open(annotations_file.as_posix(), 'r') as h:
    annotations = json.load(h)

In [14]:
annotations_images = {}
for im in annotations['images']:
    annotations_images[im['id']] = im

In [15]:
for a in annotations['annotations']:
    bbox = a['bbox']
    img_info = annotations_images[a['image_id']]
    w = img_info['width']
    h = img_info['height']
    assert 0 <= bbox[0] <= w, "Problem with {}, {}".format(a, img_info)
    assert 0 <= bbox[1] <= h, "Problem with {}, {}".format(a, img_info)
    assert 0 <= bbox[0] + bbox[2] <= w, "Problem with {}, {}".format(a, img_info)
    assert 0 <= bbox[1] + bbox[3] <= h, "Problem with {}, {}".format(a, img_info)
    if bbox[2] < 1 or bbox[3] < 1:
        print("Found zero bbox: ", a)

('Found zero bbox: ', {u'IsTruncated': 0, u'segmentation': [], u'area': 0, u'iscrowd': 0, u'IsInside': 0, u'IsOccluded': 0, u'image_id': u'330aa8283c3f543f', u'bbox': [693, 294, 39, 0], u'IsDepiction': 0, u'category_id': 42, u'id': -8855916499609862000})
('Found zero bbox: ', {u'IsTruncated': 0, u'segmentation': [], u'area': 0, u'iscrowd': 0, u'IsInside': 0, u'IsOccluded': 0, u'image_id': u'36a8935e3b98fda6', u'bbox': [191, 366, 0, 2], u'IsDepiction': 0, u'category_id': 276, u'id': -2008481524992296630})
('Found zero bbox: ', {u'IsTruncated': 0, u'segmentation': [], u'area': 0, u'iscrowd': 0, u'IsInside': 0, u'IsOccluded': 0, u'image_id': u'3a1a866065696e51', u'bbox': [699, 269, 5, 0], u'IsDepiction': 0, u'category_id': 433, u'id': -2141126454031441549})
('Found zero bbox: ', {u'IsTruncated': 0, u'segmentation': [], u'area': 0, u'iscrowd': 0, u'IsInside': 0, u'IsOccluded': 0, u'image_id': u'4e416d4066440a8f', u'bbox': [61, 313, 9, 0], u'IsDepiction': 0, u'category_id': 252, u'id': 5130

In [18]:
len(annotations['annotations']), len(annotations['images'])

(624169, 107988)

In [34]:
images_path = VALIDATION_IMAGES_PATH
annotations_path = VALIDATION_ANNOTATIONS_CSV_PATH

annotations = pd.read_csv(annotations_path, index_col="ImageID")
annotations['LabelName'] = annotations['LabelName'].map(labels_description.set_index(0)[1])

In [35]:
image_id = "a2f7ab86fb274aa0"

img = Image.open(images_path / "{}.jpg".format(image_id))

if max(img.size) > 2000 or min(img.size) < 100:
    raise RuntimeError("")

image_info = {
        "id": image_id,
        "file_name": "{}.jpg".format(image_id),
        "width": img.size[0],
        "height": img.size[1],
}    
bboxes, labels, meta = get_bboxes_labels_meta(img.size, image_id)

In [39]:
coco_annotations = []
for i, (bbox, label, m) in enumerate(zip(bboxes, labels, meta)):
    m = [int(v) for v in m]
    bbox = [int(v) for v in bbox.tolist()]
    annotation_id = hash(image_id + "_{}".format(i))
    annotation_info = {
        "id": annotation_id,
        "image_id": image_id,
        "category_id": categories[label],
        "IsOccluded": m[0],
        "IsTruncated": m[1],
        "iscrowd": m[2] if not ignore_is_crowd else 0,
        "IsDepiction": m[3],
        "IsInside": m[4],            
        "area": int(compute_area(bbox)),
        "bbox": bbox,
        "segmentation": [],
    }
    coco_annotations.append(annotation_info)

In [40]:
coco_annotations

[{'IsDepiction': 1,
  'IsInside': 0,
  'IsOccluded': 0,
  'IsTruncated': 0,
  'area': 132,
  'bbox': [127, 711, 11, 12],
  'category_id': 14,
  'id': -9011256856298810697,
  'image_id': 'a2f7ab86fb274aa0',
  'iscrowd': 0,
  'segmentation': []},
 {'IsDepiction': 1,
  'IsInside': 0,
  'IsOccluded': 0,
  'IsTruncated': 0,
  'area': 180,
  'bbox': [131, 461, 10, 18],
  'category_id': 14,
  'id': -9011256856298810698,
  'image_id': 'a2f7ab86fb274aa0',
  'iscrowd': 0,
  'segmentation': []},
 {'IsDepiction': 1,
  'IsInside': 0,
  'IsOccluded': 0,
  'IsTruncated': 0,
  'area': 96,
  'bbox': [133, 492, 8, 12],
  'category_id': 14,
  'id': -9011256856298810699,
  'image_id': 'a2f7ab86fb274aa0',
  'iscrowd': 0,
  'segmentation': []},
 {'IsDepiction': 1,
  'IsInside': 0,
  'IsOccluded': 0,
  'IsTruncated': 0,
  'area': 150,
  'bbox': [134, 738, 10, 15],
  'category_id': 14,
  'id': -9011256856298810700,
  'image_id': 'a2f7ab86fb274aa0',
  'iscrowd': 0,
  'segmentation': []},
 {'IsDepiction': 1,
  

In [12]:
import json

output_folder = Path(".").resolve().parent / "input" / "as_mscoco" / "annotations" 
annotations_file = output_folder / "train_overfit.json"

with open(annotations_file.as_posix(), 'r') as h:
    annotations = json.load(h)

In [14]:
for a in annotations['annotations']:
    bbox = a['bbox']
    img_info = [im for im in annotations['images'] if im['id'] == a['image_id']]
    assert len(img_info) == 1
    img_info = img_info[0]
    w = img_info['width']
    h = img_info['height']
    assert 0 <= bbox[0] <= w, "Problem with {}, {}".format(a, img_info)
    assert 0 <= bbox[1] <= h, "Problem with {}, {}".format(a, img_info)
    assert 0 <= bbox[0] + bbox[2] <= w, "Problem with {}, {}".format(a, img_info)
    assert 0 <= bbox[1] + bbox[3] <= h, "Problem with {}, {}".format(a, img_info)